# Transformer model - Machine Translation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Data collection and processing

In [ ]:
import spacy.cli

spacy.cli.download("en_core_web_sm")
spacy.cli.download("da_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('da_core_news_sm')


In [ ]:
import da_core_news_sm
import en_core_web_sm

spacy_da = da_core_news_sm.load()
spacy_en = en_core_web_sm.load()

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

SRC_LANGUAGE = 'da'
TGT_LANGUAGE = 'en'
# maps ordering of sentence pairs as they come in the dataset
LANGUAGE_INDEX = {SRC_LANGUAGE: 1, TGT_LANGUAGE: 0}

# Place-holders
token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='da_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

In [ ]:
from torchtext.vocab import FastText

fasttext_en = FastText(language="en")
fasttext_da = FastText(language="da")

.vector_cache/wiki.en.vec: 6.60GB [00:45, 144MB/s]                            
100%|██████████| 2519370/2519370 [03:37<00:00, 11591.25it/s]
.vector_cache/wiki.da.vec: 823MB [00:05, 145MB/s]                           
100%|██████████| 312956/312956 [00:26<00:00, 11692.96it/s]


In [ ]:
from collections import OrderedDict, Counter
from torchtext.vocab import vocab, Vectors
import io

path = './drive/My Drive/531_lab4/data/'
train_da, train_en = 'train.da', 'train.en'
valid_da, valid_en = 'dev.da', 'dev.en'
test_da, test_en = 'test.da', 'test.en'

def build_vocab(
    filepath, 
    tokenizer, 
    min_freq=1, 
    specials=["unk"], 
  ):
  """Generate vocabulary object for a given language."""
  counter = Counter()
  with open(filepath, encoding="utf-8") as f:
    for i, line in enumerate(f.readlines()):
      if i == 0:  # skip header
        continue

      out_tokens = line.strip("\n")
      counter.update(tokenizer(out_tokens))
    
    # sort and wrap as OrderedDict
    ordered = OrderedDict(sorted(counter.items(), key=lambda x: x[1], reverse=True))
    
    # build vocab object
    v_obj = vocab(
      ordered, 
      min_freq=min_freq, 
      specials=specials,
    )
    
    return v_obj

In [ ]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
SPECIAL_SYMBOLS = ['<unk>', '<pad>', '<bos>', '<eos>']
MIN_FREQ = 3

file_mapping = {
    SRC_LANGUAGE: [train_da, valid_da, test_da],
    TGT_LANGUAGE: [train_en, valid_en, test_en]
}

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:

  vocab_transform[ln] = build_vocab(
    path + file_mapping[ln][0],  # only training data
    token_transform[ln],
    min_freq=MIN_FREQ,
    specials=SPECIAL_SYMBOLS,
  )
  vocab_transform[ln].set_default_index(UNK_IDX)

In [ ]:
print(f"Unique tokens in source (da) vocabulary: {len(vocab_transform[SRC_LANGUAGE])}")
print(f"Unique tokens in target (en) vocabulary: {len(vocab_transform[TGT_LANGUAGE])}")

Unique tokens in source (da) vocabulary: 12429
Unique tokens in target (en) vocabulary: 11726


In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = {
  "train": 16,
  "valid": 128,
  "test": 128
}

def extract_transform(path, src_filename, tgt_filename):
  """Build the dataset containing paired sentences with source and target languages"""
  src_raw_iter = iter(io.open(path + src_filename, encoding="utf-8"))
  tgt_raw_iter = iter(io.open(path + tgt_filename, encoding="utf-8"))
  data = []
  for src_item, tgt_item in zip(src_raw_iter, tgt_raw_iter):
    src_raw = src_item.strip("\n")
    tgt_raw = tgt_item.strip("\n")
    data.append((src_raw, tgt_raw))

  return data

train_data = extract_transform(path, train_da, train_en)
val_data = extract_transform(path, valid_da, valid_en)
test_data = extract_transform(path, test_da, test_en)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(val_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 50000
Number of validation examples: 2000
Number of testing examples: 2000


In [ ]:
print(train_data[0])
print(val_data[0])
print(test_data[0])

('Hvordan går det med din mor?', "- how's your mom holding' up?")
('- En af os er nødt til at finde bilen!', '- One of us have to get to the car...')
('under henvisning til Rådets forordning (EF) nr. 1405/2006 af 18. september 2006 om særlige foranstaltninger på landbrugsområdet til fordel for de mindre øer i Det Ægæiske Hav og om ændring af forordning (EF) nr. 1782/2003 [1], særlig artikel 14, og', 'Having regard to Council Regulation (EC) No 1405/2006 of 18 September 2006 laying down specific measures for agriculture in favour of the smaller Aegean islands and amending Regulation (EC) No 1782/2003 [1], and in particular Article 14 thereof,')


In [ ]:
token_transform[SRC_LANGUAGE]('- En af os er nødt til at finde bilen!')

['-', 'En', 'af', 'os', 'er', 'nødt', 'til', 'at', 'finde', 'bilen', '!']

In [ ]:
token_transform[SRC_LANGUAGE]('-En af os er nødt til at finde bilen!')

['-En', 'af', 'os', 'er', 'nødt', 'til', 'at', 'finde', 'bilen', '!']

In [ ]:
import torch

# helper function to club together sequential operations
def sequential_transforms(*transforms):
  def func(txt_input):
      for transform in transforms:
          txt_input = transform(txt_input)
      return txt_input
  return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
  return torch.cat((torch.tensor([BOS_IDX]),
                    torch.tensor(token_ids),
                    torch.tensor([EOS_IDX])))

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
train_dataloader = DataLoader(
    train_data, 
    batch_size=BATCH_SIZE["train"], 
    collate_fn=collate_fn,
    shuffle=True
)
valid_dataloader = DataLoader(
    val_data, 
    batch_size=BATCH_SIZE["valid"], 
    collate_fn=collate_fn,
    shuffle=True
)
test_dataloader = DataLoader(
    test_data, 
    batch_size=BATCH_SIZE["test"], 
    collate_fn=collate_fn,
    shuffle=False
)

In [ ]:
# batch example of training data
for batch in train_dataloader:
    src, trg = batch
    print('tensor size of source language:', src.shape)
    print('tensor size of target language:', trg.shape)
    print('the tensor of first example in target language:', trg[:, 0])
    break

tensor size of source language: torch.Size([24, 16])
tensor size of target language: torch.Size([27, 16])
the tensor of first example in target language: tensor([   2,   12, 1547,  112,    0,   15,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1])


In [ ]:
# Making sure that the first example correspond to the first line in the test file
test_data[0]

('under henvisning til Rådets forordning (EF) nr. 1405/2006 af 18. september 2006 om særlige foranstaltninger på landbrugsområdet til fordel for de mindre øer i Det Ægæiske Hav og om ændring af forordning (EF) nr. 1782/2003 [1], særlig artikel 14, og',
 'Having regard to Council Regulation (EC) No 1405/2006 of 18 September 2006 laying down specific measures for agriculture in favour of the smaller Aegean islands and amending Regulation (EC) No 1782/2003 [1], and in particular Article 14 thereof,')

In [ ]:
import pickle

with open("./drive/My Drive/Colab Notebooks/ckpt_lab4/train_dataloader", "wb") as f:
     pickle.dump(train_dataloader, f)

with open("./drive/My Drive/Colab Notebooks/ckpt_lab4/valid_dataloader", "wb") as f:
     pickle.dump(valid_dataloader, f)

with open("./drive/My Drive/Colab Notebooks/ckpt_lab4/test_dataloader", "wb") as f:
     pickle.dump(test_dataloader, f)

## 2. Training

In [ ]:
from torch import Tensor
import torch.nn as nn
import math
from torch.nn import Transformer

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(
          self,
          emb_size: int,
          dropout: float,
          maxlen: int = 5000
        ):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        
        self.register_buffer('pos_embedding', pos_embedding)  # needed by Transformer module
        self.dropout = nn.Dropout(dropout)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    # def __init__(self, vocab_size: int, embedding: vocab.Vectors):
    def __init__(self, emb_size, embedding: Vectors):
        super(TokenEmbedding, self).__init__()
        #self.embedding = nn.Embedding(vocab_size, emb_size)
        self.embed = embedding
        #self.emb_size = emb_size
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embed(tokens.long()) * math.sqrt(self.emb_size)


class Seq2SeqTransformer(nn.Module):
    def __init__(
          self,
          num_encoder_layers: int,
          num_decoder_layers: int,
          emb_size: int,
          nhead: int,
          src_vocab_size: int,
          tgt_vocab_size: int,
          src_embedding: Vectors,
          tgt_embedding: Vectors,
          dim_feedforward: int = 512,
          dropout: float = 0.1
        ):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(
            d_model=emb_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        # self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        # self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.src_tok_emb = TokenEmbedding(emb_size, src_embedding)
        self.tgt_tok_emb = TokenEmbedding(emb_size, tgt_embedding)    
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(
          self,
          src: Tensor,
          trg: Tensor,
          src_mask: Tensor,
          tgt_mask: Tensor,
          src_padding_mask: Tensor,
          tgt_padding_mask: Tensor,
          memory_key_padding_mask: Tensor
        ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# FastText vectors for source and target language
src_tokens = [tok for tok, _ in vocab_transform[SRC_LANGUAGE].get_stoi().items()]
tgt_tokens = [tok for tok, _ in vocab_transform[TGT_LANGUAGE].get_stoi().items()]

src_ft_embeddings = fasttext_da.get_vecs_by_tokens(
    src_tokens, 
    lower_case_backup=True
)
src_embeddings = torch.nn.Embedding.from_pretrained(src_ft_embeddings)

tgt_ft_embeddings = fasttext_en.get_vecs_by_tokens(
    tgt_tokens, 
    lower_case_backup=True
)
tgt_embeddings = torch.nn.Embedding.from_pretrained(tgt_ft_embeddings)

print(src_embeddings)
print(tgt_embeddings)

Embedding(12429, 300)
Embedding(11726, 300)


In [ ]:
torch.manual_seed(531)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 300  # FastText dim
NHEAD = 6
FFN_HID_DIM = 1024
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                 tgt_embeddings, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_normal_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
from tqdm import tqdm

def train_epoch(model, iterator, optimizer):
    model.train()
    losses = 0

    for src, tgt in tqdm(iterator):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(iterator)


def evaluate(model, iterator):
    model.eval()
    losses = 0

    for src, tgt in iterator:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(iterator)

In [ ]:
import gc
from timeit import default_timer as timer

NUM_EPOCHS = 40

for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    train_loss = train_epoch(transformer, train_dataloader, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer, valid_dataloader)

    state_dict_model = transformer.state_dict()
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
    }

    torch.save(state, "./drive/MyDrive/Colab Notebooks/ckpt_lab4/transformer_" + str(epoch) + ".pt")

    torch.cuda.empty_cache()
    gc.collect()

    print((f"Epoch: {epoch},  "f"Epoch time = {(end_time - start_time):.3f}s"))
    print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. PPL: {math.exp(val_loss):7.3f}')

100%|██████████| 3125/3125 [01:43<00:00, 30.05it/s]


Epoch: 1,  Epoch time = 103.988s
	 Train Loss: 5.280 | Train PPL: 196.296
	 Val. Loss: 5.020 |  Val. PPL: 151.484


100%|██████████| 3125/3125 [01:43<00:00, 30.18it/s]


Epoch: 2,  Epoch time = 103.549s
	 Train Loss: 4.893 | Train PPL: 133.359
	 Val. Loss: 4.794 |  Val. PPL: 120.823


100%|██████████| 3125/3125 [01:43<00:00, 30.12it/s]


...

Epoch: 38,  Epoch time = 102.382s
	 Train Loss: 3.243 | Train PPL:  25.614
	 Val. Loss: 3.719 |  Val. PPL:  41.228


100%|██████████| 3125/3125 [01:41<00:00, 30.68it/s]


Epoch: 39,  Epoch time = 101.869s
	 Train Loss: 3.238 | Train PPL:  25.479
	 Val. Loss: 3.715 |  Val. PPL:  41.040


100%|██████████| 3125/3125 [01:41<00:00, 30.71it/s]


Epoch: 40,  Epoch time = 101.770s
	 Train Loss: 3.227 | Train PPL:  25.198
	 Val. Loss: 3.745 |  Val. PPL:  42.295


In [ ]:
transformer.load_state_dict(torch.load('./drive/MyDrive/Colab Notebooks/ckpt_lab4/transformer_30.pt')['state_dict'])
transformer = transformer.to(DEVICE)

In [ ]:
# Further training for another 10 epochs as it still has room for convergence
import gc
from timeit import default_timer as timer

NUM_EPOCHS = 10

for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    train_loss = train_epoch(transformer, train_dataloader, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer, valid_dataloader)

    state_dict_model = transformer.state_dict()
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
    }

    torch.save(state, "./drive/MyDrive/Colab Notebooks/ckpt_lab4/transformer_" + str(epoch + 20) + ".pt")

    torch.cuda.empty_cache()
    gc.collect()

    print((f"Epoch: {epoch},  "f"Epoch time = {(end_time - start_time):.3f}s"))
    print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. PPL: {math.exp(val_loss):7.3f}')

100%|██████████| 3125/3125 [02:06<00:00, 24.64it/s]


Epoch: 1,  Epoch time = 126.817s
	 Train Loss: 3.500 | Train PPL:  33.129
	 Val. Loss: 3.805 |  Val. PPL:  44.934


100%|██████████| 3125/3125 [02:03<00:00, 25.21it/s]


Epoch: 2,  Epoch time = 123.958s
	 Train Loss: 3.467 | Train PPL:  32.052
	 Val. Loss: 3.803 |  Val. PPL:  44.843


100%|██████████| 3125/3125 [02:03<00:00, 25.31it/s]


...

Epoch: 8,  Epoch time = 122.980s
	 Train Loss: 3.364 | Train PPL:  28.918
	 Val. Loss: 3.780 |  Val. PPL:  43.816


100%|██████████| 3125/3125 [02:02<00:00, 25.43it/s]


Epoch: 9,  Epoch time = 122.869s
	 Train Loss: 3.356 | Train PPL:  28.677
	 Val. Loss: 3.783 |  Val. PPL:  43.928


100%|██████████| 3125/3125 [02:02<00:00, 25.48it/s]


Epoch: 10,  Epoch time = 122.649s
	 Train Loss: 3.346 | Train PPL:  28.381
	 Val. Loss: 3.772 |  Val. PPL:  43.447


## 3. Evaluation

In [ ]:
torch.manual_seed(531)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 300 
NHEAD = 6
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 4

transformer_best = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                      NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, src_embeddings,
                                      tgt_embeddings, FFN_HID_DIM)

In [ ]:
transformer_best.load_state_dict(torch.load('./drive/MyDrive/Colab Notebooks/ckpt_lab4/transformer_36.pt')['state_dict'])
transformer_best = transformer_best.to(DEVICE)

In [ ]:
def inference(model, dataset):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    dataset: list of tuples with raw sentence pairs.

    Output:
    Corpus BLEU score.
    '''
    from nltk.translate.bleu_score import corpus_bleu

    all_translations = []
    all_references = []
    predictions_list = []
    for src, tgt in tqdm(dataset):
      out_translation = translate(model, src)
      all_translations.append(out_translation.split())
      all_references.append([token_transform[TGT_LANGUAGE](tgt)])
      predictions_list.append(out_translation)

    assert len(all_references) == len(all_translations)

    corpus_bleu_score = corpus_bleu(all_references, all_translations)  
    return corpus_bleu_score, predictions_list


# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  
        src, 
        src_mask, 
        max_len=num_tokens + 5, 
        start_symbol=BOS_IDX
    ).flatten()

    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
test_src, test_tgt = test_data[0]
out_translation = translate(transformer_best, test_src)
# out_numericalized = text_transform[TGT_LANGUAGE](out_translation)[1:-1]
# ref_numericalized = text_transform[TGT_LANGUAGE](test_tgt)
print(f"Source sentence: {test_src}")
print(f"Our system translation: {out_translation}")
print(f"Reference translation: {test_tgt}")

Source sentence: under henvisning til Rådets forordning (EF) nr. 1405/2006 af 18. september 2006 om særlige foranstaltninger på landbrugsområdet til fordel for de mindre øer i Det Ægæiske Hav og om ændring af forordning (EF) nr. 1782/2003 [1], særlig artikel 14, og
Our system translation:  Having regard to Council Regulation ( EC ) No <unk> of 20 September 2006 on the market of the market in the European Economic Community and amending Council Regulation ( EC ) No EC [ 1 ] , and in particular Article 6 thereof , 
Reference translation: Having regard to Council Regulation (EC) No 1405/2006 of 18 September 2006 laying down specific measures for agriculture in favour of the smaller Aegean islands and amending Regulation (EC) No 1782/2003 [1], and in particular Article 14 thereof,


In [ ]:
out_trans_tokenized = out_translation.split()
out_ref_tokenized = token_transform[TGT_LANGUAGE](test_tgt)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

reference = [[token_transform[TGT_LANGUAGE](test_tgt)]]
candidate = [out_translation.split()]

print('Cumulative 1-gram: %f' % corpus_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % corpus_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % corpus_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % corpus_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.695483
Cumulative 2-gram: 0.596287
Cumulative 3-gram: 0.509136
Cumulative 4-gram: 0.444876


In [ ]:
corpus_bleu_score, predictions_list = inference(transformer_best, test_data)

100%|██████████| 2000/2000 [02:57<00:00, 11.29it/s]


In [ ]:
print(corpus_bleu_score)

0.09820819618879685


In [ ]:
'''
write the translations to a file. this function will help you generate the submission file for the first question.
'''
def out_prediction(save_directory, prediction_list):
    """
    out_prediction takes four input varibles: first_name, last_name, save_directory, prediction_list
    <first_name>, string, your first name, e.g., TOM
    <last_name>, string, your last name, e.g., SMITH
    <save_directory>, string, directory to save the submission file, e.g., ./drive/My Drive/Colab Notebooks/ckpt_mt_lab4
    <prediction_list>, list of strings which includes all your predictions (or translations) of TEST samples
          e.g., ['This is the translation of my first sentence', 'This is the translation of my second sentence',...]
                    
    Generate a file is named with <yourfirstname>_<yourlastname>.txt in save directory
    """
    absolute_file_path = "{}/{}_{}.txt".format(save_directory)
    output_file = open(absolute_file_path,'w', encoding="utf-8")
    for item in prediction_list:
        output_file.write(item+"\n")
    output_file.close()
    print("submission file for the first question successfully saved at %s"%absolute_file_path)

'''
sample output:
submission file for the first question successfully saved at ./drive/My Drive/Colab Notebooks/lab4/FIRSTNAME_LASTNAME.txt
'''


'\nsample output:\nsubmission file for the first question successfully saved at ./drive/My Drive/Colab Notebooks/lab4/FIRSTNAME_LASTNAME.txt\n'

In [ ]:
outpath = "/content/drive/MyDrive/Colab Notebooks/531_lab4"
out_prediction(outpath, predictions_list)

submission file for the first question successfully saved at /content/drive/MyDrive/Colab Notebooks/531_lab4/JUAN_ROESEL_2.txt


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!perl '/content/drive/MyDrive/Colab Notebooks/531_lab4/multi-bleu.perl' '/content/drive/MyDrive/Colab Notebooks/531_lab4/data/test.en' < '/content/drive/MyDrive/Colab Notebooks/531_lab4/JUAN_ROESEL_2.txt'

BLEU = 5.41, 25.9/8.3/3.1/1.3 (BP=1.000, ratio=1.111, hyp_len=27916, ref_len=25134)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


## Reference:

* https://nlp.seas.harvard.edu/2018/04/03/attention.html#position-wise-feed-forward-networks

* https://pytorch.org/docs/master/nn.html#transformerencoderlayer

* https://pytorch.org/docs/master/_modules/torch/nn/modules/transformer.html#Transformer

* https://pytorch.org/tutorials/beginner/translation_transformer.html